# Next Paper 

In [1]:
from alarms2 import *

# Section 0: Setting Default Variables & Loading Alarm and Operator Data

In [2]:
filter_short_alarms = [20, 120]  #seconds
path = "./data/new/"

In [3]:
alarms_fname = "formatted-all-month-alarms.csv" 
operator_fname = "operator-all-month-actions.csv"

df_main_alarms = pd.read_csv(path + alarms_fname, low_memory=False ,parse_dates=["StartTime", "EndTime"])
df_main_alarms["TimeDelta"] = df_main_alarms["EndTime"] - df_main_alarms["StartTime"]
df_main_alarms["TimeDelta"] = df_main_alarms["TimeDelta"].apply(lambda arg: timedelta.total_seconds(arg)) 
df_main_alarms["Month"] = df_main_alarms["StartTime"].apply(lambda arg: arg.month)

df_main_actions = pd.read_csv(path + operator_fname, low_memory=False ,parse_dates=["EventTime"])
df_main_actions["Month"] = df_main_actions["EventTime"].apply(lambda arg: arg.month)

In [4]:
df_main_actions.info(), df_main_alarms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56656 entries, 0 to 56655
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   MachineName    56656 non-null  object        
 1   SourceName     56656 non-null  object        
 2   EventTime      56656 non-null  datetime64[ns]
 3   Severity       56656 non-null  int64         
 4   Message        56656 non-null  object        
 5   Mask           56656 non-null  int64         
 6   NewState       56656 non-null  int64         
 7   EventType      56656 non-null  object        
 8   EventCategory  56656 non-null  object        
 9   AckReq         56656 non-null  bool          
 10  ActorID        56656 non-null  object        
 11  Attributes     56656 non-null  object        
 12  Area           56656 non-null  object        
 13  Month          56656 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(4), object(8)
memory usage: 5.7+ 

(None, None)

# Section 1: Removing Chattering from Alarms Data

### use case=> Bandwidht and Storage Reduction

Retention period.

Suppose that on average each alarm takes roughly 1KB of sotorage space. 



In [5]:
# removeShortDurationAlarms(df_main_alarms)
# storageAnalysis(df_main_alarms)

# Section 2: Relating Operator Action with Alarm Data

How is this use case important?

In [6]:
# months_f = df_main_alarms["Month"].unique()
# chunks = 8
# min_intersection_f = 6
# gapf = (60 * 60)* 4 
# weightf = 2 # to remove edges in the final graph
# snames_f = ["47TI1713"]

# mg1,mg2 = getFinalOperatorAlarmRelationGraph(df_main_alarms,df_main_actions,chunks,min_intersection_f,months_f,snames_f,filter_short_alarms[1],gapf,weightf)
# print(">> Done")

In [7]:
# # assert mg1.number_of_edges() == mg2.number_of_edges()
# print(">> # of Edges Main graph1 {} and in graph2 {}".format( mg1.number_of_edges() , mg2.number_of_edges()))

# operator_nodes = [action for action in mg1.nodes if action.find("Operator")!=-1]

# node2num_neigbs = {n: len(list(mg1.neighbors(n))) for n in operator_nodes}
# t3 = [(source,mg1.nodes[source]["count"],num_neighbs) for source,num_neighbs in node2num_neigbs.items()]
# print(*[">>{},{},# of neighbours {}\n".format(*tup) for tup in sorted(t3,key=lambda arg: arg[1], reverse=True)])

# Verify the results based count with orignal csv data



In [8]:
# print(">> Building Visualization .....")
# print(g.number_of_nodes())
# nt = Network("400x", "100%", notebook=True)
# nt.from_nx(g)
# nt.options.edges.Color  = "Gray"
# nt.toggle_hide_edges_on_drag(True)
# # nt.toggle_physics(False)
# nt.repulsion()
# nt.toggle_stabilization(True)
# nt.show_buttons(filter_=['physics','edges'])
# # nt.toggle_ph
# nt.show("nt.html")

# Section 3: Finding the most important alarms in the data. 

In [16]:
months_f = df_main_alarms["Month"].unique()
chunks = 20
min_intersection_f = 4
gapf = 60*2 
snames_f = ["47TI1713"]

mg1,mg2 = getFinalAlarmRelationsGraph(df_main_alarms,chunks,min_intersection_f,months_f,snames_f,filter_short_alarms[1],gapf)

print(">> # of Edges Main graph1 {} and in graph2 {}".format( mg1.number_of_edges() , mg2.number_of_edges()))
print(">> Done")

>> Starting to find relations between alarms....
>> Number of sub graphs to be constructed : 20
>> Filter: Only Include these months:  [3 4 5 6]
>> Filter: Gap Filter for prev to next alarm = 120s.
>> Filter: Ingore SourceNames  ['47TI1713']
        ----------------------------------
>> Index Range = (0, 2742), Min & Max dates = (datetime.date(2019, 3, 6), datetime.date(2019, 3, 10))
>> # of nodes = 152, # of edges = 475
        ----------------------------------
>> Index Range = (2742, 5484), Min & Max dates = (datetime.date(2019, 3, 10), datetime.date(2019, 3, 15))
>> # of nodes = 156, # of edges = 715
        ----------------------------------
>> Index Range = (5484, 8226), Min & Max dates = (datetime.date(2019, 3, 15), datetime.date(2019, 3, 19))
>> # of nodes = 170, # of edges = 727
        ----------------------------------
>> Index Range = (8226, 10968), Min & Max dates = (datetime.date(2019, 3, 19), datetime.date(2019, 4, 3))
>> # of nodes = 168, # of edges = 647
        ------

In [17]:
# Google PageRank Algo
result = nx.algorithms.link_analysis.pagerank_alg.pagerank(mg1,weight="weight",max_iter=100000)
result = {k:float(format(v, '.4f')) for k,v in sorted(result.items(), key=lambda arg: arg[1], reverse=True)}
print(">> Page Rank (Highest to Lowest) :",list(result.items())[:50])

print("              --------------------------------------------------")

# Eigenvector Centrality Algo
result = nx.eigenvector_centrality(mg1, weight="weight")
result = {k:float(format(v, '.4f')) for k,v in sorted(result.items(), key=lambda arg: arg[1], reverse=True)}
print(">> Eigenvector Centrality (Highest to Lowest) :",list(result.items())[:50])


>> Page Rank (Highest to Lowest) : [('48PI2025', 0.1157), ('47PIC1755', 0.0449), ('47PI1734', 0.0418), ('47LIC005', 0.0412), ('47FI1568', 0.035), ('47LI003', 0.0273), ('47PDI1746', 0.0246), ('48FIX2040', 0.0243), ('48TIC2027', 0.0241), ('47TI003A', 0.019), ('48XL002-ANN', 0.0177), ('47TI1512C', 0.0162), ('47TI003B', 0.0154), ('SIO-18XA001', 0.0154), ('47TIC3520', 0.0141), ('47TAL003AB-ANN', 0.014), ('47FD-036H-ANN', 0.0139), ('47FD-036HH-ANN', 0.0139), ('48XL001-ANN', 0.0108), ('47_AMPSC_PMCC1L', 0.0102), ('47TI1512B', 0.01), ('47_AMPSA_PMCC1L', 0.0099), ('47TI1512ABC', 0.0099), ('47TXI1592', 0.0098), ('48FI2021A', 0.0093), ('47FI1575', 0.0088), ('48FALL2021-ANN', 0.0084), ('47PI2031A', 0.0081), ('47PI2031ABC', 0.0081), ('47TI414H', 0.0081), ('47TI414G', 0.0081), ('47FIC1524', 0.0081), ('47FIC1524_CALX', 0.0081), ('47PIC027', 0.0075), ('47PDI1747', 0.0072), ('19LI105A', 0.0071), ('47TI1723', 0.0065), ('47PI2039C', 0.0062), ('47FI043', 0.0061), ('47PIC2023', 0.0061), ('47XL015-ANN', 0.0

In [18]:
# Hits Algo
h, a = nx.hits(mg1)
result = h
result = {k:float(format(v, '.4f')) for k,v in sorted(result.items(), key=lambda arg: arg[1], reverse=True)}
print(">> Hub => Outgoing Edges: Based on out_degree (max to min):",list(result.items())[:50])

print("              --------------------------------------------------")

result = a
result = {k:float(format(v, '.4f')) for k,v in sorted(result.items(), key=lambda arg: arg[1], reverse=True)}
print(">> Auth => Incoming Edges: Based on in_degree (max to min):",list(result.items())[:50])


>> Hub => Outgoing Edges: Based on out_degree (max to min): [('47XL016-ANN', 0.2226), ('47XL015-ANN', 0.2173), ('47PI1734', 0.0445), ('48PI2025', 0.042), ('47FLI-039', 0.039), ('48XL002-ANN', 0.0356), ('48XL001-ANN', 0.0336), ('SIO-18XA001', 0.0332), ('47PIC1755', 0.0271), ('47TIC3520', 0.0184), ('47PIC1505', 0.0183), ('47TI003B', 0.0156), ('48TIC034', 0.0147), ('47FLI-042', 0.0146), ('47PIC2033A', 0.0129), ('47FI1568', 0.0127), ('47PDI1746', 0.0123), ('47LI003', 0.0115), ('47LIC005', 0.0098), ('47TI003A', 0.0091), ('48LI015', 0.0088), ('48FI2003A', 0.0086), ('47TI1769', 0.008), ('47PIC3522', 0.0065), ('48TIC2027', 0.0063), ('48FIX2040', 0.0062), ('48LI013', 0.0061), ('19LI105A', 0.0059), ('47FI1005', 0.0057), ('47TAL003AB-ANN', 0.0055), ('47TI3519', 0.0053), ('47LIC009', 0.0046), ('47FI043', 0.004), ('47TI002C', 0.0036), ('47LIC1509', 0.0033), ('48XL010-ANN', 0.0031), ('48XL009-ANN', 0.003), ('47TI1512A', 0.0029), ('48LI011', 0.0028), ('48FALL2003-ANN', 0.0027), ('47FLI-037', 0.0026),

In [12]:
# Degree Analysis
nodes_dict = {}
for s in list(mg1.nodes):
    nodes_dict[s] = {"count":mg1.nodes[s]["count"], "outd" : mg1.out_degree(s,"weight"), "ind":mg1.in_degree(s,"weight"), "totald": mg1.degree(s,"weight")}

nodes_dict = {k:v for k, v in sorted(nodes_dict.items(), key=lambda arg: arg[1]["ind"], reverse=True) if v["count"]>4 and v["totald"]>1}
print(nodes_dict)

{'48PI2025': {'count': 5036, 'outd': 992, 'ind': 1926, 'totald': 2918}, '47LI003': {'count': 1016, 'outd': 201, 'ind': 1295, 'totald': 1496}, '47PIC1755': {'count': 2455, 'outd': 446, 'ind': 908, 'totald': 1354}, '47PI1734': {'count': 3208, 'outd': 614, 'ind': 887, 'totald': 1501}, '47LIC005': {'count': 1720, 'outd': 234, 'ind': 765, 'totald': 999}, '47FI1568': {'count': 1843, 'outd': 250, 'ind': 698, 'totald': 948}, '47TI1512C': {'count': 191, 'outd': 25, 'ind': 627, 'totald': 652}, '19G402-ANN': {'count': 592, 'outd': 29, 'ind': 595, 'totald': 624}, 'SIO-18XA001': {'count': 1943, 'outd': 625, 'ind': 493, 'totald': 1118}, '47PDI1746': {'count': 1408, 'outd': 250, 'ind': 485, 'totald': 735}, '48XL002-ANN': {'count': 1844, 'outd': 870, 'ind': 425, 'totald': 1295}, '47TI1512B': {'count': 221, 'outd': 340, 'ind': 406, 'totald': 746}, '47TI1512ABC': {'count': 209, 'outd': 287, 'ind': 406, 'totald': 693}, '47TIC3520': {'count': 954, 'outd': 364, 'ind': 406, 'totald': 770}, '47XL015-ANN': {'

In [19]:
for edge in mg1.edges:
    mg1.edges[edge]["value"] = mg1.edges[edge]["weight"]
    mg1.edges[edge]["title"] = "{},{}".format(edge,mg1.edges[edge]["weight"])

for node in mg1.nodes:
    mg1.nodes[node]["title"] = "{}, {}".format(node,mg1.nodes[node]['count']) 


nt = Network("400x", "100%", notebook=True)
nt.from_nx(mg1)
nt.options.edges.Color  = "Gray"
nt.toggle_hide_edges_on_drag(True)
# nt.toggle_physics(False)
nt.repulsion()
nt.toggle_stabilization(True)
nt.show_buttons(filter_=['physics','edges'])
# nt.toggle_ph
nt.show("nt.html")
# visualize_graph(mg1)

# Extras


### Per Sensor Based Analysis